# Introduction to Spark

## Outline

-RDD's
-Create
-Map, FlatMap
-Reduce

### Create RDDs

[RDD Documentation](http://spark.apache.org/docs/0.7.0/api/pyspark/pyspark.rdd.RDD-class.html)

The `parallelize` method is a utility for initializing RDDs.


### Transformations and Actions

**Actions** return values

- `collect`
- `reduce`
- `take`
- `count`

**Transformations** return pointers to new RDDs

- `map`, `flatmap`
- `reduceByKey`
- `filter`
- `glom`

Everything starts with a `SparkContext`

In [14]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

# do something to prove it worksC
rdd = sc.parallelize(range(1000))
rdd.takeSample(False, 5)


[987, 51, 34, 70, 754]

In [15]:
#range(start, end=None, step=1, numSlices=None)

rdd = sc.parallelize(range(0, 100, 1), 5)
print (rdd.collect())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


What does this look like?

- `glom`: Returns an RDD list from each partition of an RDD.
- `collect`: Returns a list from all elements of an RDD to the DRIVER.

In [16]:
for x in rdd.glom().collect():
    print(x) 

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]
[40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]
[60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79]
[80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


Use map and square each element of the RDD

In [17]:
#two transformations - A DAG with two tasks
mappedRdd = (rdd.map(lambda x: x*x)
                 .filter(lambda x : x > 1000))
#one action
print (mappedRdd.collect())


[1024, 1089, 1156, 1225, 1296, 1369, 1444, 1521, 1600, 1681, 1764, 1849, 1936, 2025, 2116, 2209, 2304, 2401, 2500, 2601, 2704, 2809, 2916, 3025, 3136, 3249, 3364, 3481, 3600, 3721, 3844, 3969, 4096, 4225, 4356, 4489, 4624, 4761, 4900, 5041, 5184, 5329, 5476, 5625, 5776, 5929, 6084, 6241, 6400, 6561, 6724, 6889, 7056, 7225, 7396, 7569, 7744, 7921, 8100, 8281, 8464, 8649, 8836, 9025, 9216, 9409, 9604, 9801]


Use reduce to sum the squares

In [18]:
reduced = mappedRdd.reduce(lambda x, y: x+y)
print (type(reduced))
print (reduced)


<class 'int'>
317934


In [19]:
#Calling reduce on an empty list fails
emptyList =[0]
rdd = sc.parallelize(emptyList)
reduced = rdd.reduce(lambda x,y: x+y)
print (reduced)

0


### `map` and `Flatmap`

Create a new RDD  - a list of lists

In [20]:
rdd = sc.parallelize([ [2, 3, 4],[0, 1],[5, 6, 7, 8] ])
rdd.collect()

[[2, 3, 4], [0, 1], [5, 6, 7, 8]]

Notice the type of each element mapped is a list

Pull off the first element of each list

In [21]:
elementTypes = rdd.map(lambda x: type(x))
print (elementTypes.collect())
firstElement = rdd.map(lambda x: x[0])
print (firstElement.collect())

[<class 'list'>, <class 'list'>, <class 'list'>]
[2, 0, 5]


The list element of the list alist is alist[-1:]

print the last element of each sublist

In [22]:
lastElement = rdd.map(lambda x: x[-1])
print (lastElement.collect())

[4, 1, 8]


In [23]:
lastElement = rdd.map(lambda x: x[-1:])
print (lastElement.collect())

[[4], [1], [8]]


Return a new RDD by first applying a function (the length of each string), then flatten.

In [24]:
y = rdd.map(lambda x: range(len(x))).collect()
print (y)

[range(0, 3), range(0, 2), range(0, 4)]


Or I can flatten the results...

In [25]:
z = rdd.flatMap(lambda x: range(len(x))).collect()
print (z)

[0, 1, 2, 0, 1, 0, 1, 2, 3]


Or flatten the original results

In [26]:
rdd.flatMap(lambda x: x).collect()

[2, 3, 4, 0, 1, 5, 6, 7, 8]

### Reduction

In [27]:
rdd = sc.parallelize(range(1000), 5)

In [28]:
rdd.reduce(lambda x,y: x+y)

499500

In [29]:
rdd = sc.parallelize([ [2, 3, 4],[0, 1],[5, 6, 7, 8] ])

In [30]:
rdd.flatMap(lambda x: x).reduce(lambda x,y: x+y)

36

 ### Python Tuples and Key Value Pairs
 

In [31]:
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 2)])
rdd.collect()

[('a', 1), ('b', 1), ('a', 2)]

In [32]:
#print the type of each element above
rddType = rdd.map(lambda x: type(x))
print (rddType.collect())

[<class 'tuple'>, <class 'tuple'>, <class 'tuple'>]


In [33]:
first = rdd.map(lambda x: x[0])
print (first.collect())


['a', 'b', 'a']


In [34]:
print(rdd.reduceByKey(lambda x,y: x+y).collect())

[('b', 1), ('a', 3)]


In [35]:
rdd.groupByKey().mapValues(lambda x: sum(x)).collect()

[('b', 1), ('a', 3)]

In [36]:
rdd = sc.parallelize([("hamlet", 1), ("claudius", 1), ("hamlet", 1)])

In [37]:
rdd.countByKey()

defaultdict(int, {'hamlet': 2, 'claudius': 1})

In [38]:
rdd.reduceByKey(lambda x,y: x+y).collect()

[('claudius', 1), ('hamlet', 2)]